In [7]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNetV2
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [61]:
base_model=MobileNetV2(weights='imagenet',include_top=False, input_shape= (224,224,3)) #imports the mobilenet model and discards the last 1000 neuron layer.
base_model.trainable = False
inp = keras.layers.Input(shape=(224,224,3))
x = keras.layers.experimental.preprocessing.Rescaling(
    1/255.
)(inp)

x=base_model(x)
x=GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(5,activation='softmax')(x) #final layer with softmax activation

In [62]:
model=Model(inputs = inp,outputs=preds)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['acc'])

In [63]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
rescaling_6 (Rescaling)      (None, 224, 224, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_4 ( (None, 1280)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               655872    
_________________________________________________________________
dense_9 (Dense)              (None, 5)                 2565      
Total params: 2,916,421
Trainable params: 658,437
Non-trainable params: 2,257,984
___________________________________________

In [64]:
img_dir = "/content/drive/MyDrive/Datasets/DATASET"
batch_size=32
image_size=(224, 224)

datagen_cat = ImageDataGenerator(
   
)

train_gen_cat = datagen_cat.flow_from_directory(
    img_dir+"/TRAIN",
    seed=1337,
    shuffle=True,
    target_size=image_size,
    batch_size=batch_size,
)

val_gen_cat = datagen_cat.flow_from_directory(
    img_dir+"/TEST",
    seed=1337,
    shuffle=True,
    target_size=image_size,
    batch_size=batch_size,
)

Found 1081 images belonging to 5 classes.
Found 470 images belonging to 5 classes.


In [65]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
Patience=2
BestModelWeightsPath = "{}_weights.best.h5".format(model)
checkpoint = ModelCheckpoint(
    BestModelWeightsPath, 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True, 
    mode='max', 
    save_weights_only = True,
)
reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_acc', 
    factor=0.2, 
    patience=Patience, 
    verbose=1, 
    mode='max', 
    cooldown=2, 
    min_lr=0.000001
)
early = EarlyStopping(
    monitor="val_acc", 
    mode="max", 
    patience=Patience*2,
    restore_best_weights=True
)
callbacks_list_final= [checkpoint, early, reduceLROnPlat]

In [66]:
classification = model.fit(train_gen_cat,validation_data=val_gen_cat,epochs=100,verbose=1,callbacks=callbacks_list_final)

Epoch 1/100
 2/34 [>.............................] - ETA: 10s - loss: 2.0149 - acc: 0.2804 

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


34/34 [==============================] - ETA: 0s - loss: 1.2111 - acc: 0.5763

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:770: UserWarning: Possibly corrupt EXIF data.  Expecting to read 11 bytes but only got 10. Skipping tag 42037
  " Skipping tag %s" % (size, len(data), tag)


34/34 [==============================] - 38s 1s/step - loss: 1.1994 - acc: 0.5804 - val_loss: 0.2407 - val_acc: 0.9234

Epoch 00001: val_acc improved from -inf to 0.92340, saving model to <tensorflow.python.keras.engine.functional.Functional object at 0x7fab8bbce6d0>_weights.best.h5
Epoch 2/100
34/34 [==============================] - 34s 1000ms/step - loss: 0.2393 - acc: 0.9355 - val_loss: 0.2073 - val_acc: 0.9362

Epoch 00002: val_acc improved from 0.92340 to 0.93617, saving model to <tensorflow.python.keras.engine.functional.Functional object at 0x7fab8bbce6d0>_weights.best.h5
Epoch 3/100
34/34 [==============================] - 33s 993ms/step - loss: 0.1373 - acc: 0.9734 - val_loss: 0.1838 - val_acc: 0.9468

Epoch 00003: val_acc improved from 0.93617 to 0.94681, saving model to <tensorflow.python.keras.engine.functional.Functional object at 0x7fab8bbce6d0>_weights.best.h5
Epoch 4/100
34/34 [==============================] - 34s 1s/step - loss: 0.0608 - acc: 0.9960 - val_loss: 0.185

In [70]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmppztl696m/assets


INFO:tensorflow:Assets written to: /tmp/tmppztl696m/assets


In [69]:
import numpy as np
import tensorflow as tf
import cv2
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
img = cv2.imread("/content/drive/MyDrive/Datasets/DATASET/TEST/tree/00000006.jpg")
img = cv2.resize(img, (224,224))
# print(img)
print(input_details)
input_data = np.array(np.expand_dims(img, axis = 0), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[{'name': 'input_19', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[[2.1032670e-06 1.6492051e-04 2.0010867e-07 9.9901330e-01 8.1944861e-04]]
